In [1]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from   torch import nn
from   torch.nn import functional as F
from   torch.utils.data import TensorDataset, DataLoader

In [2]:
data = pd.read_csv(r'dataset/coin_Bitcoin.csv')
data.head()

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,Bitcoin,BTC,2013-04-29 23:59:59,147.488007,134.000000,134.444000,144.539993,0.0,1.603769e+09
1,2,Bitcoin,BTC,2013-04-30 23:59:59,146.929993,134.050003,144.000000,139.000000,0.0,1.542813e+09
2,3,Bitcoin,BTC,2013-05-01 23:59:59,139.889999,107.720001,139.000000,116.989998,0.0,1.298955e+09
3,4,Bitcoin,BTC,2013-05-02 23:59:59,125.599998,92.281898,116.379997,105.209999,0.0,1.168517e+09
4,5,Bitcoin,BTC,2013-05-03 23:59:59,108.127998,79.099998,106.250000,97.750000,0.0,1.085995e+09


In [3]:
data.select_dtypes(exclude=object).corrwith(data['Marketcap'])

SNo          0.664713
High         0.999094
Low          0.999072
Open         0.998382
Close        0.999709
Volume       0.809156
Marketcap    1.000000
dtype: float64

In [4]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.input_   = nn.Linear(in_features=6, out_features=50)
        self.hidden_1 = nn.Linear(in_features=50, out_features=25)
        self.hidden_2 = nn.Linear(in_features=25, out_features=15)
        self.hidden_3 = nn.Linear(in_features=15, out_features=8)
        self.output_  = nn.Linear(in_features=8, out_features=1)

        self._init_weights(self.modules)
    
    def forward(self, x):
        x = F.relu(self.input_(x))
        x = F.relu(self.hidden_1(x))
        x = F.relu(self.hidden_2(x))
        x = F.relu(self.hidden_3(x))
        x = self.output_(x)
        return x
    

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, nonlinearity='elu')
            if module.bias is not None:
                module.bias.data.zero_()

In [5]:
model = Model()
model.parameters

<bound method Module.parameters of Model(
  (input_): Linear(in_features=6, out_features=50, bias=True)
  (hidden_1): Linear(in_features=50, out_features=25, bias=True)
  (hidden_2): Linear(in_features=25, out_features=15, bias=True)
  (hidden_3): Linear(in_features=15, out_features=8, bias=True)
  (output_): Linear(in_features=8, out_features=1, bias=True)
)>

In [6]:
X = data.drop(columns=['Marketcap','Name','Symbol','Date']).values
y = data['Marketcap'].values

In [7]:
X = StandardScaler().fit_transform(X)

In [8]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.log1p(torch.tensor(y, dtype=torch.float32))

In [9]:
X.shape

torch.Size([2991, 6])

In [10]:
tensor =  TensorDataset(X, y)

In [11]:
tensor[0]

(tensor([-1.7315, -0.5795, -0.5845, -0.5817, -0.5813, -0.5775]),
 tensor(21.1956))

In [12]:
train = DataLoader(dataset=tensor, batch_size=20, shuffle=True, drop_last=True)

In [13]:
loss = nn.MSELoss()

In [14]:
optimizer = torch.optim.SGD(
    params= model.parameters(), momentum=0.99, nesterov=True
    )

In [15]:
for epoch in range(100):
    for xtrain, ytrain in train:
        pred = model(xtrain)

        loss_r = loss(ytrain, pred.T[0])   
        loss_r.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f'epoch {epoch} ------- loss {loss_r.item()}')

epoch 0 ------- loss 6.307467937469482
epoch 1 ------- loss 2.5705037117004395
epoch 2 ------- loss 3.05842661857605
epoch 3 ------- loss 1.0619616508483887
epoch 4 ------- loss 0.6717609763145447
epoch 5 ------- loss 0.4898086190223694
epoch 6 ------- loss 1.154835820198059
epoch 7 ------- loss 0.41202640533447266
epoch 8 ------- loss 0.6342685222625732
epoch 9 ------- loss 1.0455719232559204
epoch 10 ------- loss 0.934607982635498
epoch 11 ------- loss 0.5893527865409851
epoch 12 ------- loss 0.40977582335472107
epoch 13 ------- loss 1.4815428256988525
epoch 14 ------- loss 0.6045101881027222
epoch 15 ------- loss 0.5758377313613892
epoch 16 ------- loss 1.0614264011383057
epoch 17 ------- loss 0.6474952101707458
epoch 18 ------- loss 0.5557147264480591
epoch 19 ------- loss 0.5202887058258057
epoch 20 ------- loss 0.22874435782432556
epoch 21 ------- loss 1.5485475063323975
epoch 22 ------- loss 0.624991774559021
epoch 23 ------- loss 0.6675430536270142
epoch 24 ------- loss 2.18328